In [1]:
import os
os.chdir('..')

In [2]:
from eva5_final.model import Model
from eva5_final.dataset import ComboDataset
from eva5_final.config import Config
from eva5_final.yolov3.utils.datasets import LoadImagesAndLabels
import pytorch_lightning as pl
import torch
import matplotlib.pyplot as plt

In [3]:
train_dataset = ComboDataset(Config)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = Config.BATCH_SIZE,
    num_workers = 4,
    shuffle = not Config.yolo_config.opt.rect,
    pin_memory = True,
    collate_fn = train_dataset.collate_fn
)

val_dataset = ComboDataset(Config, train=False)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size = Config.BATCH_SIZE,
    num_workers = 4,
    pin_memory = True,
    collate_fn = val_dataset.collate_fn
)


Caching labels data/yolo/labels.npy (3034 found, 0 missing, 169 empty, 0 duplicate, for 3203 images): 100%|██████████| 3203/3203 [00:00<00:00, 13417.74it/s]
Caching labels data/yolo/labels.npy (297 found, 14 missing, 7 empty, 0 duplicate, for 318 images): 100%|██████████| 318/318 [00:00<00:00, 1717.56it/s]


In [4]:
model = Model(Config, train_dataset.yolo_labels, len(train_dataset) / Config.EPOCHS)

Using cache found in /home/ouroboros/.cache/torch/hub/facebookresearch_WSL-Images_master
Loading weights:  weights/midas.pt
Model Summary: 147 layers, 5.93877e+07 parameters, 5.93877e+07 gradients


In [5]:
trainer = pl.Trainer(max_epochs=Config.EPOCHS)
trainer.fit(
    model,
    train_dataloader=train_loader,
    val_dataloaders=val_loader
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name          | Type     | Params
-------------------------------------------
0 | midas_net     | MidasNet | 105 M 
1 | yolo_head     | YoloHead | 5 M   
2 | yolo_detector | Darknet  | 59 M  
Epoch 0:   2%|▏         | 4/221 [00:28<25:34,  7.07s/it, loss=6.338, v_num=8, yolo loss=6.05]


1